# [Day 17: Trick Shot](https://adventofcode.com/2021/day/17)

In [1]:
import dataclasses as dc
import re

# Notes

__min and max for `vx`__
 * there is no direct relation between `x` and `y`
 * `x` is only influenced by drag: it will always get to `0`
 * max: launching with a `vx` which will overshoot the x_right of the area in one step does never make sense: __max vx = x_right of the area__
 * the minimal `vx` does not depend on `vy` but has to be simulated

__min and max for `vy`__
 * `y` is not influenced by drag
 * for the minimum: launching with a `vy` which will overshoot y_bottom of the area in one shoot never makes sense: __min vy = y_bottom of the area__
 * when shooting up: the `y=0` line will always be crossed with `vy=-(vy+1)`, so to prevent overshoot `vy` has a maximum of `-(y_bottom + 1)`

## Part 1

In [2]:
example_data_part1 = {
    "target_area": (20, -5, 30, -10),  # x1, y1, x2, y2
    "launches": [  # v_x, v_y, max_y, expected_hit
        (7, 2, 3, True),
        (6, 3, 6, True),
        (9, 0, 0, True),
        (6, 9, 45, True),
        (17, -4, 0, False),
    ]
}

In [3]:
@dc.dataclass
class Probe:
    vx: int
    vy: int
    x: int = 0
    y: int = 0
    max_y: int = 0


    def next_step(self):
        # adjust x
        self.x += self.vx
        if self.vx != 0:
            if self.vx < 0:
                self.vx += 1
            else:
                self.vx -= 1

        # adjust y
        self.y += self.vy
        self.vy -= 1
        
        # track max_y
        if self.y > self.max_y:
            self.max_y = self.y

            
    def simulate(self, x_left, y_top, x_right, y_bottom):
        missed = self.y < y_bottom
        while not missed:
            self.next_step()
            in_target_area = (x_left <= self.x <= x_right) and (y_top >= self.y >= y_bottom)
            if in_target_area:
                return True, self.max_y, self.x, self.y
            else:
                missed = (self.y < y_bottom) or (self.x > x_right)
        return False, self.max_y, self.x, self.y

In [4]:
for v_x, v_y, max_y, expected_hit in example_data_part1["launches"]:
    probe = Probe(v_x, v_y)
    hit, max_y, x, y = probe.simulate(*example_data_part1["target_area"])
    print(f"Check simulate {v_x, v_y} -> {max_y}: {hit == expected_hit}")

Check simulate (7, 2) -> 3: True
Check simulate (6, 3) -> 6: True
Check simulate (9, 0) -> 0: True
Check simulate (6, 9) -> 45: True
Check simulate (17, -4) -> 0: True


In [5]:
def brute_force_simulate(x1, y1, x2, y2):
    x_left, x_right = (x1, x2) if x1 < x2 else (x2, x1)
    y_top, y_bottom = (y1, y2) if y1 > y2 else (y2, y1)

    # maximal v_x is equal to x_right (otherwise it will overshoot)
    max_vx = x_right

    # minimal v_x is the v_x required to travel x_left distance
    for min_vx in range(1, x_left):
        x, vx = 0, min_vx
        hit, max_y, x, y = Probe(vx, 0).simulate(x_left, y_top, x_right, y_bottom)
        if x >= x_left:
            break
    
    # search within v_x range for best v_y
    best_vx, best_vy, best_max_y, n_hits = 0, 0, 0, 0
    for vx in range(min_vx, max_vx+1):
        for vy in range(y_bottom, -y_bottom):
            hit, max_y, x, y = Probe(vx, vy).simulate(x_left, y_top, x_right, y_bottom)
            if max_y > best_max_y:
                best_vx, best_vy, best_max_y = vx, vy, max_y
            if hit:
                n_hits += 1

    return best_vx, best_vy, best_max_y, n_hits

In [6]:
vx, vy, max_y, _ = brute_force_simulate(20, -5, 30, -10)
print(f"Check part 1: {(vx, vy, max_y) == (6, 9, 45)}")

Check part 1: True


In [7]:
with open(r"..\data\Day 17 input.txt", "r") as fh_in:
    input_data = fh_in.readline().strip()
    # target area: x=48..70, y=-189..-148
    area = tuple(map(int, re.match(r".*x\=([-+]?\d+)\.{2}([-+]?\d+).*y\=([-+]?\d+)\.{2}([-+]?\d+)", input_data).groups()))
    area = (area[0], area[2], area[1], area[3])
print(f"Input check: {len(area) == 4}")

Input check: True


In [8]:
vx, vy, max_y, _ = brute_force_simulate(*area)
print(f"Answer part 1: {max_y}")

Answer part 1: 17766


## Part 2

In [9]:
vx, vy, max_y, n_hits = brute_force_simulate(20, -5, 30, -10)
print(f"Check part 2: {(vx, vy, max_y, n_hits) == (6, 9, 45, 112)}")

Check part 2: True


In [10]:
vx, vy, max_y, n_hits = brute_force_simulate(*area)
print(f"Answer part 2: {n_hits}")

Answer part 2: 1733
